Loading the Data

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load the data
df_0 = pd.read_csv(r"C:\Users\pingk\Downloads\fadhli nitip\ffrt46_3a_clnd_blncd.csv")

In [3]:
df_0

,prov_char,sample_code,649.893,650.376,650.858,651.34,651.822,652.304,652.786,653.268,...,3998.194,3998.676,3999.158,3999.64,4000.122,tgp_name,dgp_name,fgp_name,country_name,thnoth_name
0,KBX,ID-KBX-068-2306-012,0.014445,0.013811,0.013240,0.012696,0.012224,0.011944,0.011994,0.012458,...,0.000827,0.000632,0.000437,0.000263,0.000141,Group 3,Group 2,Group 4,Indonesia,Non-Thai
1,RIX,ID-RIX-070-2312-006,0.014224,0.014795,0.015192,0.015504,0.015866,0.016407,0.017193,0.018182,...,-0.001422,-0.001488,-0.001552,-0.001625,-0.001707,Group 2,Group 2,Group 3,Indonesia,Non-Thai
2,KBX,ID-KBX-068-2307-024,0.016033,0.017364,0.018503,0.019348,0.019843,0.019982,0.019800,0.019370,...,0.002876,0.002779,0.002675,0.002569,0.002470,Group 3,Group 2,Group 4,Indonesia,Non-Thai
3,RIX,ID-RIX-094-2401-001,0.021065,0.020825,0.020055,0.018751,0.016976,0.014865,0.012592,0.010367,...,0.003455,0.003512,0.003532,0.003505,0.003432,Group 2,Group 2,Group 3,Indonesia,Non-Thai
4,KBX,ID-KBX-068-2306-007,0.016785,0.018009,0.019500,0.021119,0.022708,0.024123,0.025268,0.026096,...,0.002037,0.001959,0.001903,0.001874,0.001867,Group 3,Group 2,Group 4,Indonesia,Non-Thai
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1847,SNI,TH-SNI-019-2311-003,0.015161,0.015769,0.016838,0.018318,0.019935,0.021252,0.021853,0.021451,...,0.004020,0.003986,0.003984,0.003987,0.003959,Group 2,Group 2,Group 3,Thailand,Thai
1848,KBI,TH-KBI-101-2208-001,0.018566,0.017833,0.016916,0.015912,0.014965,0.014228,0.013810,0.013752,...,0.000929,0.000985,0.001014,0.001001,0.000939,Group 2,Group 2,Group 2,Thailand,Thai
1849,CPN,TH-CPN-005-2311-002,0.005516,0.003654,0.002713,0.002845,0.004042,0.006150,0.008935,0.012103,...,0.001960,0.001965,0.001949,0.001920,0.001878,Group 2,Group 2,Group 2,Thailand,Thai
1850,PTN,TH-PTN-101-2208-002,0.015483,0.015174,0.015231,0.015658,0.016369,0.017202,0.017983,0.018555,...,0.000589,0.000586,0.000572,0.000547,0.000515,Group 2,Group 1,Group 2,Thailand,Thai


Selecting Regions of Interest

In [4]:
# Define the regions of interest
regions_of_interest = [
    (2996, 3016), (2943, 2963), (2912, 2932), (2843, 2863),
    (1490, 2010), (1455, 1475), (1408, 1428), (1368, 1388),
    (1225, 1245), (1150, 1170), (1106, 1126), (1088, 1108),
    (990, 920), (712, 732)
]

# Extract columns corresponding to the regions of interest
columns_to_focus = []
for start, end in regions_of_interest:
    columns_to_focus.extend([col for col in df_0.columns[4:-5] if start <= float(col) <= end])

# Create a new DataFrame with the selected regions
df_0_selected_regions = df_0[columns_to_focus]

# Combine the selected regions with the target column and other relevant columns
df_0_selected_regions = pd.concat([df_0[['country_name', 'prov_char']], df_0_selected_regions], axis=1)

# Save the DataFrame for further processing
df_0_selected_regions.to_csv('data/data file 3/data_1.csv', index=False)


FUNCTIONS

Baseline Correction

In [5]:
from scipy.signal import savgol_filter

# Function for baseline correction with dynamic window length
def baseline_correction(spectrum, default_window_length=15, polyorder=3):
    spectrum_length = len(spectrum)
    if spectrum_length < default_window_length:
        window_length = spectrum_length // 2 * 2 + 1  # Make window length odd and less than the size of the spectrum
    else:
        window_length = default_window_length
    baseline = savgol_filter(spectrum, window_length, polyorder, mode='nearest')
    corrected_spectrum = spectrum - baseline
    return corrected_spectrum

# Apply baseline correction
df_baseline_corrected_v0 = df_0_selected_regions.copy()
for col in columns_to_focus:
    df_baseline_corrected_v0[col] = baseline_correction(df_baseline_corrected_v0[col])

# Save the baseline corrected data
df_baseline_corrected_v0.to_csv('data/data file 3/data_1_bslcrct.csv', index=False)

SavGol Smoothing

In [6]:
# Function for Savitzky-Golay smoothing
def savitzky_golay_smoothing(spectrum, default_window_length=11, polyorder=2):
    window_length = min(default_window_length, len(spectrum) // 2 * 2 + 1)  # Make window length odd and less than or equal to the size of the spectrum
    if window_length < 3:  # Ensure window length is at least 3
        window_length = 3
    return savgol_filter(spectrum, window_length, polyorder, mode='nearest')  # Set mode to 'nearest'

# Apply smoothing
df_smoothed_v0 = df_baseline_corrected_v0.copy()
for col in columns_to_focus:
    df_smoothed_v0[col] = savitzky_golay_smoothing(df_smoothed_v0[col])

# Save the smoothed data
df_smoothed_v0.to_csv('data/data file 3/data_1_smoothed.csv', index=False)


Normalization

In [7]:
# Function for normalization (Min-Max scaling)
def min_max_normalization(spectrum):
    return (spectrum - np.min(spectrum)) / (np.max(spectrum) - np.min(spectrum))

# Apply normalization
df_normalized_v0 = df_smoothed_v0.copy()
for col in columns_to_focus:
    df_normalized_v0[col] = min_max_normalization(df_normalized_v0[col])

# Save the normalized data
df_normalized_v0.to_csv('data/data file 3/data_1_normalized.csv', index=False)


Derivatization (np.gradient)

In [8]:
# Calculate the first derivative using np.gradient
data_spectrum = df_normalized_v0.iloc[:, 2:].values
first_derivative_np = np.gradient(data_spectrum, axis=1)

# Calculate the second derivative using np.gradient
second_derivative_np = np.gradient(first_derivative_np, axis=1)

# Convert the results back to DataFrame
data_1_der_np = pd.DataFrame(first_derivative_np, columns=df_normalized_v0.columns[2:])
data_2_der_np = pd.DataFrame(second_derivative_np, columns=df_normalized_v0.columns[2:])

# Combine the first two columns from the original dataset with the np.gradient derivatives

# Extract the first two columns
first_two_columns = df_normalized_v0.iloc[:, :2]

# Combine the first two columns with the derivatives
data_1_der_combined = pd.concat([first_two_columns, data_1_der_np], axis=1)
data_2_der_combined = pd.concat([first_two_columns, data_2_der_np], axis=1)

# Export the combined data to CSV
data_1_der_combined.to_csv('data/data file 3/data_1_1_der.csv', index=False)
data_2_der_combined.to_csv('data/data file 3/data_1_2_der.csv', index=False)

Derivatization (SavGol)

In [9]:
# Extract the spectrum data
data_spectrum = df_normalized_v0.iloc[:, 2:].values

# Apply Savitzky-Golay filter for the first derivative
first_derivative_savgol = savgol_filter(data_spectrum, window_length=5, polyorder=2, deriv=1, axis=1)

# Apply Savitzky-Golay filter for the second derivative
second_derivative_savgol = savgol_filter(data_spectrum, window_length=5, polyorder=2, deriv=2, axis=1)

# Convert the results back to DataFrame
data_1_der_savgol = pd.DataFrame(first_derivative_savgol, columns=df_normalized_v0.columns[2:])
data_2_der_savgol = pd.DataFrame(second_derivative_savgol, columns=df_normalized_v0.columns[2:])

# Extract the first two columns
first_two_columns = df_normalized_v0.iloc[:, :2]

# Combine the first two columns with the Savitzky-Golay derivatives
data_1_der_savgol_combined = pd.concat([first_two_columns, data_1_der_savgol], axis=1)
data_2_der_savgol_combined = pd.concat([first_two_columns, data_2_der_savgol], axis=1)

# Export the combined data to CSV
data_1_der_savgol_combined.to_csv('data/data file 3/data_1_1_der_savgol.csv', index=False)
data_2_der_savgol_combined.to_csv('data/data file 3/data_1_2_der_savgol.csv', index=False)

SNV

In [10]:
def snv(spectrum):
    return (spectrum - np.mean(spectrum)) / np.std(spectrum)

# Apply SNV to the selected regions
df_snv = df_0_selected_regions.copy()
for col in columns_to_focus:
    df_snv[col] = snv(df_snv[col])

# Save the SNV data
df_snv.to_csv('data/data file 3/data_1_snv.csv', index=False)


c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\numpy\_core\fromnumeric.py:3800: FutureWarning: The behavior of DataFrame.std with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return std(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)


Random Normal Variate (RNV)

In [11]:
def rnv(spectrum):
    random_noise = np.random.normal(0, np.std(spectrum), spectrum.shape)
    return spectrum + random_noise

# Apply RNV to the selected regions
df_rnv = df_0_selected_regions.copy()
for col in columns_to_focus:
    df_rnv[col] = rnv(df_rnv[col])

# Save the RNV data
df_rnv.to_csv('data/data file 3/data_1_rnv.csv', index=False)


Multiplicative Scatter Correction (MSC)

In [12]:
# def msc(spectrum, reference):
#     mean_spectrum = np.mean(reference, axis=0)
#     fit = np.polyfit(mean_spectrum, spectrum, 1, full=True)
#     corrected_spectrum = (spectrum - fit[0][1]) / fit[0][0]
#     return corrected_spectrum

# # Apply MSC to the selected regions
# df_msc = df_0_selected_regions.copy()
# for col in columns_to_focus:
#     df_msc[col] = msc(df_msc[col])

# # Save the MSC data
# df_msc.to_csv('data/data file 3/data_1_msc2.csv', index=False)


In [13]:
# Multiplicative Scatter Correction (MSC) function
def msc(input_data):
    # Mean center the data
    mean_spectrum = np.mean(input_data, axis=0)
    input_data_centered = input_data - mean_spectrum

    # Perform MSC
    reference = np.mean(input_data, axis=0)
    msc_data = np.zeros_like(input_data)

    for i in range(input_data.shape[0]):
        fit = np.polyfit(reference, input_data_centered[i, :], 1, full=True)
        msc_data[i, :] = (input_data_centered[i, :] - fit[0][1]) / fit[0][0]
    
    return msc_data

# Extract the spectral data from the dataframe
spectral_data = df_0_selected_regions.iloc[:, 2:].values

# Apply MSC
msc_corrected_data = msc(spectral_data)

# Create a new dataframe with the MSC corrected data
msc_df = pd.DataFrame(msc_corrected_data, columns=df_0_selected_regions.columns[2:])
msc_df.insert(0, 'prov_char', df_0_selected_regions['prov_char'])
msc_df.insert(0, 'country_name', df_0_selected_regions['country_name'])

# Save the MSC corrected data to a new CSV file
msc_corrected_file_path = 'data/data file 3/data_1_msc_corrected.csv'
msc_df.to_csv(msc_corrected_file_path, index=False)


Classification and Evaluation (40-fold)

In [14]:
# Define the target variable
target = 'country_name'

# Ensure columns_to_focus are correctly identified
numeric_cols_df_0 = df_0_selected_regions.select_dtypes(include=[np.number]).columns.tolist()
columns_to_focus = numeric_cols_df_0  # Ensure columns are correctly selected

# Classification and evaluation function using 40-fold CV
def classify_and_evaluate(df, columns):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import cross_val_score
    from sklearn.preprocessing import LabelEncoder

    # Encode target variable
    le = LabelEncoder()
    y = le.fit_transform(df[target])

    # Define features
    X = df[columns]

    # Initialize the classifier
    model = RandomForestClassifier(n_estimators=100, random_state=42)

    # Perform cross-validation
    scores = cross_val_score(model, X, y, cv=40)

    # Print the results
    print(f'Cross-Validation Accuracy: {np.mean(scores)}')


In [15]:

# Evaluate each preprocessing method
print("Evaluation for No Preprocessing:")
classify_and_evaluate(df_0_selected_regions, columns_to_focus)

Evaluation for No Preprocessing:
Cross-Validation Accuracy: 0.8244449583718779


In [16]:
print("Evaluation for Baseline Correction:")
df_baseline_corrected_v0z = pd.read_csv('data/data file 3/data_1_bslcrct.csv')
classify_and_evaluate(df_baseline_corrected_v0z, columns_to_focus)

Evaluation for Baseline Correction:
Cross-Validation Accuracy: 0.6976295097132285


In [17]:
print("Evaluation for Smoothing:")
df_smoothed_v0z = pd.read_csv('data/data file 3/data_1_smoothed.csv')
classify_and_evaluate(df_smoothed_v0z, columns_to_focus)

Evaluation for Smoothing:
Cross-Validation Accuracy: 0.6772432932469934


In [18]:
print("Evaluation for Normalization:")
df_normalized_v0z = pd.read_csv('data/data file 3/data_1_normalized.csv')
classify_and_evaluate(df_normalized_v0z, columns_to_focus)

Evaluation for Normalization:
Cross-Validation Accuracy: 0.6626965772432932


In [19]:
print("Evaluation for 1-Derivative Spectroscopy:")
data_1_der_combined_v0z = pd.read_csv('data/data file 3/data_1_1_der.csv')
classify_and_evaluate(data_1_der_combined_v0z, columns_to_focus)

Evaluation for 1-Derivative Spectroscopy:
Cross-Validation Accuracy: 0.7163852913968547


In [20]:
print("Evaluation for 2-Derivative Spectroscopy:")
data_2_der_combined_v0z = pd.read_csv('data/data file 3/data_1_2_der.csv')
classify_and_evaluate(data_2_der_combined_v0z, columns_to_focus)

Evaluation for 2-Derivative Spectroscopy:
Cross-Validation Accuracy: 0.7222710453283996


In [21]:
print("Evaluation for 1-SG-Derivative Spectroscopy:")
data_1_der_savgol_combined_v0z = pd.read_csv('data/data file 3/data_1_1_der_savgol.csv')
classify_and_evaluate(data_1_der_savgol_combined_v0z, columns_to_focus)

Evaluation for 1-SG-Derivative Spectroscopy:
Cross-Validation Accuracy: 0.7164893617021277


In [22]:
print("Evaluation for 2-SG-Derivative Spectroscopy:")
data_1_der_savgol_combined_v0z = pd.read_csv('data/data file 3/data_1_2_der_savgol.csv')
classify_and_evaluate(data_1_der_savgol_combined_v0z, columns_to_focus)

Evaluation for 2-SG-Derivative Spectroscopy:
Cross-Validation Accuracy: 0.7154486586493987


In [23]:
print("Evaluation for SNV:")
df_snv_v0z = pd.read_csv('data/data file 3/data_1_snv.csv')
classify_and_evaluate(df_snv_v0z, columns_to_focus)

print("Evaluation for RNV:")
df_rnv_v0z = pd.read_csv('data/data file 3/data_1_rnv.csv')
classify_and_evaluate(df_rnv_v0z, columns_to_focus)

print("Evaluation for MSC:")
df_msc_v0z = pd.read_csv('data/data file 3/data_1_msc_corrected.csv')
classify_and_evaluate(df_msc_v0z, columns_to_focus)

Evaluation for SNV:
Cross-Validation Accuracy: 0.8233926919518965
Evaluation for RNV:
Cross-Validation Accuracy: 0.6112858464384828
Evaluation for MSC:
Cross-Validation Accuracy: 0.777509250693802


Classification and Evaluation (40-fold) -with extra detailed

In [24]:
# Define the target variable
target = 'country_name'

# Ensure columns_to_focus are correctly identified
numeric_cols_df_0 = df_0_selected_regions.select_dtypes(include=[np.number]).columns.tolist()
columns_to_focus = numeric_cols_df_0  # Ensure columns are correctly selected

# Classification and evaluation function using 40-fold CV with detailed metrics
def classify_and_evaluate(df, columns):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import cross_val_score, StratifiedKFold
    from sklearn.preprocessing import LabelEncoder
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
    import numpy as np

    # Encode target variable
    le = LabelEncoder()
    y = le.fit_transform(df[target])

    # Define features
    X = df[columns]

    # Initialize the classifier
    model = RandomForestClassifier(n_estimators=100, random_state=42)

    # Initialize Stratified K-Fold Cross-Validation
    skf = StratifiedKFold(n_splits=40)

    # Arrays to store results
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    conf_matrices = []

    # Perform cross-validation
    for train_idx, test_idx in skf.split(X, y):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # Train the model
        model.fit(X_train, y_train)

        # Predict and evaluate
        y_pred = model.predict(X_test)
        accuracies.append(accuracy_score(y_test, y_pred))
        precisions.append(precision_score(y_test, y_pred, average=None))
        recalls.append(recall_score(y_test, y_pred, average=None))
        f1_scores.append(f1_score(y_test, y_pred, average=None))
        conf_matrices.append(confusion_matrix(y_test, y_pred))

    # Calculate mean scores
    mean_accuracy = np.mean(accuracies)
    mean_precision = np.mean(precisions, axis=0)
    mean_recall = np.mean(recalls, axis=0)
    mean_f1 = np.mean(f1_scores, axis=0)
    mean_conf_matrix = np.mean(conf_matrices, axis=0)

    # Print the results
    print(f'Cross-Validation Accuracy: {mean_accuracy}')
    print(f'Precision per class: {mean_precision}')
    print(f'Recall per class: {mean_recall}')
    print(f'F1-score per class: {mean_f1}')


In [25]:
# Evaluate each preprocessing method
print("Evaluation for No Preprocessing:")
classify_and_evaluate(df_0_selected_regions, columns_to_focus)

print("Evaluation for Baseline Correction:")
df_baseline_corrected_v0y = pd.read_csv('data/data file 3/data_1_bslcrct.csv')
classify_and_evaluate(df_baseline_corrected_v0y, columns_to_focus)

print("Evaluation for Smoothing:")
df_smoothed_v0y = pd.read_csv('data/data file 3/data_1_smoothed.csv')
classify_and_evaluate(df_smoothed_v0y, columns_to_focus)

print("Evaluation for Normalization:")
df_normalized_v0y = pd.read_csv('data/data file 3/data_1_normalized.csv')
classify_and_evaluate(df_normalized_v0y, columns_to_focus)

print("Evaluation for 1-Derivative Spectroscopy:")
df_derivative_v0y = pd.read_csv('data/data file 3/data_1_1_der.csv')
classify_and_evaluate(df_derivative_v0y, columns_to_focus)

print("Evaluation for 2-Derivative Spectroscopy:")
data_2_der_combined_v0y = pd.read_csv('data/data file 3/data_1_2_der.csv')
classify_and_evaluate(data_2_der_combined_v0y, columns_to_focus)

print("Evaluation for 1-SG-Derivative Spectroscopy:")
data_1_der_savgol_combined_v0y = pd.read_csv('data/data file 3/data_1_1_der_savgol.csv')
classify_and_evaluate(data_1_der_savgol_combined_v0y, columns_to_focus)

print("Evaluation for 2-SG-Derivative Spectroscopy:")
data_1_der_savgol_combined_v0y = pd.read_csv('data/data file 3/data_1_2_der_savgol.csv')
classify_and_evaluate(data_1_der_savgol_combined_v0y, columns_to_focus)

Evaluation for No Preprocessing:


c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicte

Cross-Validation Accuracy: 0.8244449583718779
Precision per class: [0.81899378 0.15       0.83492751]
Recall per class: [0.83106061 0.075      0.87663043]
F1-score per class: [0.82258612 0.1        0.85337911]
Evaluation for Baseline Correction:


c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicte

Cross-Validation Accuracy: 0.6976295097132285
Precision per class: [0.7199531  0.         0.68895528]
Recall per class: [0.65286797 0.         0.79388587]
F1-score per class: [0.68044301 0.         0.7352996 ]
Evaluation for Smoothing:


...

c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicte

Cross-Validation Accuracy: 0.7164893617021277
Precision per class: [0.7184557  0.         0.72504216]
Recall per class: [0.69366883 0.         0.79365942]
F1-score per class: [0.69979362 0.         0.7535742 ]
Evaluation for 2-SG-Derivative Spectroscopy:


c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicte

Cross-Validation Accuracy: 0.7154486586493987
Precision per class: [0.71180717 0.         0.72627957]
Recall per class: [0.70205628 0.         0.78410326]
F1-score per class: [0.70195408 0.         0.75081961]


c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [26]:
print("Evaluation for SNV:")
df_snv_v0y = pd.read_csv('data/data file 3/data_1_snv.csv')
classify_and_evaluate(df_snv_v0y, columns_to_focus)

print("Evaluation for RNV:")
df_rnv_v0y = pd.read_csv('data/data file 3/data_1_rnv.csv')
classify_and_evaluate(df_rnv_v0y, columns_to_focus)

print("Evaluation for MSC:")
df_msc_v0y = pd.read_csv('data/data file 3/data_1_msc_corrected.csv')
classify_and_evaluate(df_msc_v0y, columns_to_focus)

Evaluation for SNV:


c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicte

Cross-Validation Accuracy: 0.8233926919518965
Precision per class: [0.82619134 0.225      0.82766224]
Recall per class: [0.82186147 0.1375     0.87912138]
F1-score per class: [0.82075407 0.16666667 0.84996703]
Evaluation for RNV:


c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicte

Cross-Validation Accuracy: 0.6112858464384828
Precision per class: [0.62412203 0.         0.60740474]
Recall per class: [0.49859307 0.         0.76358696]
F1-score per class: [0.5496562  0.         0.67467787]
Evaluation for MSC:


c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicte

Cross-Validation Accuracy: 0.777509250693802
Precision per class: [0.7875538  0.         0.77977851]
Recall per class: [0.7737013  0.         0.84230072]
F1-score per class: [0.77634094 0.         0.80689716]


c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Classification and Evaluation using LOGO-CV

In [27]:
# Classification and evaluation function using LOGO-CV with detailed metrics
def classify_and_evaluate_logo_cv_detailed(df, columns):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import LeaveOneGroupOut
    from sklearn.preprocessing import LabelEncoder
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
    
    # Encode target variable
    le = LabelEncoder()
    y = le.fit_transform(df[target])
    
    # Define features
    X = df[columns]
    
    # Initialize the classifier
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    
    # Initialize LOGO-CV
    logo = LeaveOneGroupOut()
    groups = df['prov_char']
    
    # Arrays to store results
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    all_y_test = []
    all_y_pred = []
    
    # Perform LOGO-CV
    for train_idx, test_idx in logo.split(X, y, groups=groups):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        # Train the model
        model.fit(X_train, y_train)
        
        # Predict and evaluate
        y_pred = model.predict(X_test)
        accuracies.append(accuracy_score(y_test, y_pred))
        all_y_test.extend(y_test)
        all_y_pred.extend(y_pred)
        
    # Calculate overall metrics
    mean_accuracy = np.mean(accuracies)
    precision = precision_score(all_y_test, all_y_pred, average=None)
    recall = recall_score(all_y_test, all_y_pred, average=None)
    f1 = f1_score(all_y_test, all_y_pred, average=None)
    conf_matrix = confusion_matrix(all_y_test, all_y_pred)
    
    # Print the results
    print(f'Mean Accuracy: {mean_accuracy}')
    print(f'Precision per class: {precision}')
    print(f'Recall per class: {recall}')
    print(f'F1-score per class: {f1}')
    
    # Return confusion matrix for presentation
    return conf_matrix

In [28]:
# Evaluate each preprocessing method using LOGO-CV with detailed metrics
print("LOGO-CV Evaluation for No Preprocessing:")
conf_matrix_no_preprocessing_v0x = classify_and_evaluate_logo_cv_detailed(df_0_selected_regions, columns_to_focus)

print("LOGO-CV Evaluation for Baseline Correction:")
df_baseline_corrected_v0x = pd.read_csv('data/data file 3/data_1_bslcrct.csv')
conf_matrix_baseline_v0x = classify_and_evaluate_logo_cv_detailed(df_baseline_corrected_v0x, columns_to_focus)

print("LOGO-CV Evaluation for Smoothing:")
df_smoothed_v0x = pd.read_csv('data/data file 3/data_1_smoothed.csv')
conf_matrix_smoothing_v0x = classify_and_evaluate_logo_cv_detailed(df_smoothed_v0x, columns_to_focus)

print("LOGO-CV Evaluation for Normalization:")
df_normalized_v0x = pd.read_csv('data/data file 3/data_1_normalized.csv')
conf_matrix_normalization_v0x = classify_and_evaluate_logo_cv_detailed(df_normalized_v0x, columns_to_focus)

print("LOGO-CV Evaluation for 1-Derivative Spectroscopy:")
df_1_derivative_v0x = pd.read_csv('data/data file 3/data_1_1_der.csv')
conf_matrix_1_derivative_v0x = classify_and_evaluate_logo_cv_detailed(df_1_derivative_v0x, columns_to_focus)

print("LOGO-CV Evaluation for 2-Derivative Spectroscopy:")
df_2_derivative_v0x = pd.read_csv('data/data file 3/data_1_2_der.csv')
conf_matrix_2_derivative_v0x = classify_and_evaluate_logo_cv_detailed(df_2_derivative_v0x, columns_to_focus)

print("LOGO-CV Evaluation for 1-SG-Derivative Spectroscopy:")
df_1_der_savgol_combined_v0y = pd.read_csv('data/data file 3/data_1_1_der_savgol.csv')
conf_matrix_1_sg_v0x = classify_and_evaluate_logo_cv_detailed(df_1_der_savgol_combined_v0y, columns_to_focus)

print("LOGO-CV Evaluation for 2-SG-Derivative Spectroscopy:")
df_2_der_savgol_combined_v0y = pd.read_csv('data/data file 3/data_1_2_der_savgol.csv')
conf_matrix_2_sg_v0x = classify_and_evaluate_logo_cv_detailed(df_2_der_savgol_combined_v0y, columns_to_focus)

print("LOGO-CV Evaluation for SNV:")
df_snv_v0x = pd.read_csv('data/data file 3/data_1_snv.csv')
conf_matrix_snv_v0x = classify_and_evaluate_logo_cv_detailed(df_snv_v0x, columns_to_focus)

print("LOGO-CV Evaluation for RNV:")
df_rnv_v0x = pd.read_csv('data/data file 3/data_1_rnv.csv')
conf_matrix_rnv_v0x = classify_and_evaluate_logo_cv_detailed(df_rnv_v0x, columns_to_focus)

print("LOGO-CV Evaluation for MSC:")
df_msc_v0x = pd.read_csv('data/data file 3/data_1_msc_corrected.csv')
conf_matrix_msc_v0x = classify_and_evaluate_logo_cv_detailed(df_msc_v0x, columns_to_focus)

LOGO-CV Evaluation for No Preprocessing:
Mean Accuracy: 0.48605887862495284
Precision per class: [0.23263889 0.         0.42255759]
Recall per class: [0.15709261 0.         0.57451404]
F1-score per class: [0.18754374 0.         0.48695652]
LOGO-CV Evaluation for Baseline Correction:
Mean Accuracy: 0.47400980190733866
Precision per class: [0.31241084 0.         0.41325196]
Recall per class: [0.25674091 0.         0.51187905]
F1-score per class: [0.28185328 0.         0.45730825]
LOGO-CV Evaluation for Smoothing:
Mean Accuracy: 0.7050926359472643
Precision per class: [0.60492041 0.         0.60258621]
Recall per class: [0.49003517 0.         0.75485961]
F1-score per class: [0.54145078 0.         0.67018217]
LOGO-CV Evaluation for Normalization:
Mean Accuracy: 0.6880259849521804
Precision per class: [0.61102832 0.         0.6       ]
Recall per class: [0.48065651 0.         0.76457883]
F1-score per class: [0.53805774 0.         0.67236467]
LOGO-CV Evaluation for 1-Derivative Spectroscopy:

c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Mean Accuracy: 0.7305071576082358
Precision per class: [0.63046544 0.         0.6264217 ]
Recall per class: [0.52403283 0.         0.77321814]
F1-score per class: [0.57234315 0.         0.6921218 ]
LOGO-CV Evaluation for 2-Derivative Spectroscopy:


c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Mean Accuracy: 0.7363133294192997
Precision per class: [0.62695035 0.         0.62249346]
Recall per class: [0.51817116 0.         0.77105832]
F1-score per class: [0.56739409 0.         0.68885673]
LOGO-CV Evaluation for 1-SG-Derivative Spectroscopy:


...

c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Mean Accuracy: 0.7053841889578872
Precision per class: [0.62429379 0.         0.62237762]
Recall per class: [0.51817116 0.         0.76889849]
F1-score per class: [0.56630365 0.         0.68792271]
LOGO-CV Evaluation for SNV:
Mean Accuracy: 0.49454333767826836
Precision per class: [0.21172023 0.         0.43253662]
Recall per class: [0.13130129 0.         0.60583153]
F1-score per class: [0.16208394 0.         0.50472335]
LOGO-CV Evaluation for RNV:


c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Mean Accuracy: 0.46992992083363794
Precision per class: [0.171875   0.         0.36056106]
Recall per class: [0.12895662 0.         0.47192225]
F1-score per class: [0.14735432 0.         0.40879326]
LOGO-CV Evaluation for MSC:
Mean Accuracy: 0.49058094674281894
Precision per class: [0.21880065 0.         0.39271255]
Recall per class: [0.15826495 0.         0.5237581 ]
F1-score per class: [0.18367347 0.         0.44886627]


c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Display Confusion Matrices

In [29]:
# Function to display confusion matrix in a tabular format
def display_confusion_matrix(conf_matrix, class_labels):
    df_cm = pd.DataFrame(conf_matrix, index=class_labels, columns=class_labels)
    print(df_cm)

# Ensure LabelEncoder is defined and class labels are set
from sklearn.preprocessing import LabelEncoder

# Define the target variable and fit LabelEncoder
target = 'country_name'
le = LabelEncoder()
le.fit(df_0_selected_regions[target])
class_labels = le.classes_

In [30]:
# Display confusion matrices for each preprocessing method
print("Confusion Matrix for No Preprocessing:")
display_confusion_matrix(conf_matrix_no_preprocessing_v0x, class_labels)

print("Confusion Matrix for Baseline Correction:")
display_confusion_matrix(conf_matrix_baseline_v0x, class_labels)

print("Confusion Matrix for Smoothing:")
display_confusion_matrix(conf_matrix_smoothing_v0x, class_labels)

print("Confusion Matrix for Normalization:")
display_confusion_matrix(conf_matrix_normalization_v0x, class_labels)

print("Confusion Matrix for Derivative Spectroscopy:")
display_confusion_matrix(conf_matrix_1_derivative_v0x, class_labels)

print("Confusion Matrix for Derivative Spectroscopy:")
display_confusion_matrix(conf_matrix_2_derivative_v0x, class_labels)

print("Confusion Matrix for Derivative Spectroscopy:")
display_confusion_matrix(conf_matrix_1_sg_v0x, class_labels)

print("Confusion Matrix for Derivative Spectroscopy:")
display_confusion_matrix(conf_matrix_2_sg_v0x, class_labels)

print("Confusion Matrix for SNV:")
display_confusion_matrix(conf_matrix_snv_v0x, class_labels)

print("Confusion Matrix for RNV:")
display_confusion_matrix(conf_matrix_rnv_v0x, class_labels)

print("Confusion Matrix for MSC:")
display_confusion_matrix(conf_matrix_msc_v0x, class_labels)

Confusion Matrix for No Preprocessing:
           Indonesia  Malaysia  Thailand
Indonesia        134        17       702
Malaysia          48         0        25
Thailand         394         0       532
Confusion Matrix for Baseline Correction:
           Indonesia  Malaysia  Thailand
Indonesia        219         4       630
Malaysia          30         0        43
Thailand         452         0       474
Confusion Matrix for Smoothing:
           Indonesia  Malaysia  Thailand
Indonesia        418         1       434
Malaysia          46         0        27
Thailand         227         0       699
Confusion Matrix for Normalization:
           Indonesia  Malaysia  Thailand
Indonesia        410         1       442
Malaysia          43         0        30
Thailand         218         0       708
Confusion Matrix for Derivative Spectroscopy:
           Indonesia  Malaysia  Thailand
Indonesia        447         0       406
Malaysia          52         0        21
Thailand         210      

In [32]:
def snv(spectrum):
    return (spectrum - np.mean(spectrum)) / np.std(spectrum)


# Apply SNV to the selected regions
df_snv_1 = df_smoothed_v0x.copy()
for col in columns_to_focus:
    df_snv_1[col] = snv(df_snv_1[col])

# Save the SNV data
#df_snv_1.to_csv('data/data file 4/data_1_snv.csv', index=False)


In [33]:
print("Evaluation for Smooth + SNV:")
classify_and_evaluate(df_snv_1, columns_to_focus)

Evaluation for Smooth + SNV:


c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicte

Cross-Validation Accuracy: 0.6621646623496762
Precision per class: [0.67201269 0.         0.66558286]
Recall per class: [0.59312771 0.         0.778125  ]
F1-score per class: [0.62131667 0.         0.71245503]


c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
